## Ejercicio 7: Obtención de valores optimos de Hiperparámetros con GridSearchCV

Scikit-Learn proporciona la clase GridSearchCV para la selección de valores óptimos para un conjunto de hiperparámetros.

El siguiente programa obtiene los valores óptimos de los hiperparámetros para un clasificador k-NN (KNeighborsClassifier).

El clasificador se pasa como un parámetro a GridSearchCV indicando en la llamada el tipo de metrica (en nuestro caso es minkowski).
Necesitamos tambien un registro con los rangos de valores a probar.

Probaremos con un rango para n_neighbors (número de vecinos a considerar) de 1 a 19.\
Para weights estableceremos dos opciones ['uniform', 'distance'], en el primer caso todas las observaciones cuentan lo mismo para determinar la clase, en el segundo la clase de cada muestra se pondera según el inverso de la distancia para que cuenten más las más cercanas.

NOTA: el campo weights determina las distintas maneras en las que se van a ponderar los pesos de las muestras durante el entrenamiento.\

Necesitamos tambien un campo p si estamos utilizando Minkowski que determina el tipo de calculo de distancia.\
En este caso, probamos con valores 1 (distancia de Manhattan), 2 (distancia euclídea), y valores 10 y 20, con los que la distancia se aproxíma a la de Chebyshev.


Para clasificación utilizaremos el accuracy rate o tasa de aciertos.

Para la validación cruzada establecemos cv=5, lo que supone seccionar el dataset en 5 grupos.

Se valida con 1/5=20% de los datos después de haber entrenado con el 80% restante. 

Según la documentación, ahora no es necesario utilizar random_state.

Para cada combinación de valores, se realiza una validación cruzada. 

Finalmente, se muestran los detalles del clasificador óptimo, es decir, aquel para el que se obtiene el 
mejor resultado después de la validación cruzada. 



-El clasificador óptimo correspondiente lo tenemos en el atributo best_estimator_

-Los valores óptimos para los hiperparámetros en best_params_

-El valor de la tasa de aciertos o accuracy score en best_score_.


--------------\
NORMALIZACIÓN:\
Las caracteristicas predictoras se normalizan. Puesto que este es un algoritmo basado en calculo de distancias, podría ocurrir que unas caracerísticas se impusieran a otras cuando los rangos de valores no fueran comparables.
Si las características tienen rangos muy diferentes, las de mayor rango dominarán los cálculos, haciendo que el modelo dé menos importancia a las características de menor rango.

Por ejemplo:

Edad: Rango de 0 a 100 años.
Ingresos: Rango de 0 a 100,000 €.
Si no se normalizan, los ingresos tendrán más peso en el modelo simplemente por su magnitud.

-------------



In [25]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [26]:
# Creamos el DataFrame y añadimos etiquetas a las columnas
df = pd.read_csv('./data/iris.csv', names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])


In [27]:
# Obtenemos el Dataframe x con las features predictoras y el Dataframe y con la columna objetivo
x = df.drop(['class'], axis=1)
y = df['class']


In [28]:
# Normalizar valores de atributos predictores en un rango de 0 a 1. Porque es un algoritmo basado en distancia.

x = (x-np.min(x)) / (np.max(x)-np.min(x))

In [29]:
# Conjunto de valores para probar para cada uno de los hiperparámetros

param_dist = {
  'n_neighbors':        np.arange(1, 20),        # k (Número de vecinos)
  'weights':            ['uniform', 'distance'], # Ponderación de pesos segun distancia
  "p":                  [1, 2, 10, 20]           # p para distancia de Minkowski
}

grid_clf = GridSearchCV(estimator = KNeighborsClassifier(metric='minkowski'), param_grid = param_dist, cv = 5)

grid_clf.fit(x, y) # Entrenamos el clasificador

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                         'p': [1, 2, 10, 20],
                         'weights': ['uniform', 'distance']})

In [30]:
# Aquí obtenemos el clasificador optimizado
clf = grid_clf.best_estimator_

In [31]:
# Valores optimos estimados
print(grid_clf.best_params_)

{'n_neighbors': 6, 'p': 10, 'weights': 'uniform'}


In [32]:
# Tasa de aciertos
print(grid_clf.best_score_)

0.9866666666666667


Ejercicio:

Repite la ejecución del programa pero ahora sin normalizar los datos y compara los resultados. Discute el resultado de la comparativa.